In [2]:
!pip -q install langchain
!pip -q install torch
!pip -q install faiss-cpu
!pip -q install huggingface-hub
!pip -q install accelerate
!pip -q install sentence_transformers
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install gradio==3.48.0
!pip -q install pandasai[langchain]
!pip -q install pypdf
!pip -q install llama-cpp-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from getpass import getpass
import gradio as gr
import os
from langchain.embeddings import HuggingFaceEmbeddings
from pandasai import Agent
from google.colab import drive
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationSummaryBufferMemory
import io
import contextlib
import pandas as pd

In [5]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [7]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [8]:
docs=[]
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/RAG")
docs = loader.load()

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=50)
text_chunks= text_splitter.split_documents(docs)

In [11]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [13]:
repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1"

In [14]:
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.01, "max_new_tokens": 2048})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [23]:
import gradio as gr

# Function to handle the question input and return the answer
def ask_question(question):
    result = qa_chain({"question": question})
    answer = result['result'].split('Helpful Answer: ')[-1]
    return answer

# Create a Gradio interface
iface = gr.Interface(
    fn=ask_question,
    inputs="text",  # User can input a text question
    outputs="text",  # Outputs a text answer
    title="Question Answering with LangChain",
    description="Ask a question and get an answer using RetrievalQA.",
    examples=["What is the story about?", "Explain the main concept?"]
)

# Launch the interface
iface.launch()

IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://87be283f16b3fdb1e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
